In [2]:
import sys
import numpy as np
sys.path.append('../lib/')
from dbConnect import *
from kiwoomMain import *
import sqlalchemy
import pandas as pd

C:\Users\trimu\Miniconda3\envs\py36_32\lib\site-packages\pymysql\cursors.py:165: Warning: (1366, "Incorrect string value: '\\xB4\\xEB\\xC7\\xD1\\xB9\\xCE...' for column 'VARIABLE_VALUE' at row 481")
  result = self._query(query)


In [3]:
#period_years = set(['2015', '2016', '2017'])      # 2015, 2016, 2017  3 개년 

df_all = pd.read_sql("select * from financesheet where item_code='dart_OperatingIncomeLoss'", con=engine)  # 영업이익만 select

In [4]:
len(set(df_all[(df_all['smarket'] == '1')]['code']))     # 유가증권시장 종목수

740

In [5]:
len(set(df_all[(df_all['smarket'] == '2')]['code']))     # 코스닥시장 종목수

1218

In [6]:
df_all['year'] = df_all['closing_date'].str[:4]                     # year column 추가

loss_codes = df_all[df_all['amount'] < 0].code.tolist()      # 영업이익 하나라도 적자인 회사 코드 list

In [7]:
len(set(loss_codes))  # 유가증권시장 + 코스닥 영업이익 1 년이라도 적자인 회사 수

792

In [8]:
len(set(df_all[(df_all['amount'] < 0) & (df_all['smarket'] == '1')]['code']))    # 유가증권시장에서 1 년이라도 적자난 회사 수

215

In [9]:
len(set(df_all[(df_all['amount'] < 0) & (df_all['smarket'] == '2')]['code']))    # 코스닥에서 1 년이라도 적자난 회사 수

577

In [10]:
df_all = df_all[~df_all['code'].isin(loss_codes)]                 # 영업이익 하나라도 적자인 회사 제외하면 3 개년 모두 흑자인 회사만 남음

df_all.drop(columns=['item_code', 'item_name', 'closing_date'],inplace=True)    # 불필요한 두개 column 삭제

In [11]:
df_all.shape

(5730, 6)

In [12]:
df2015 = df_all.loc[df_all.year == '2015', :]                    # 연도별 영업이익을 한 row 로 merge 하기 위해 3 개 dataframe 으로 분리
df2016 = df_all.loc[df_all.year == '2016', :]
df2017 = df_all.loc[df_all.year == '2017', :]

In [13]:
df_all = pd.merge(df2015, df2016, on=['kind','smarket','code','code_name'])    
df_all = pd.merge(df_all, df2017,on=['kind','smarket','code','code_name'])

In [14]:
df_all.tail(2)

,kind,smarket,code,code_name,amount_x,year_x,amount_y,year_y,amount,year
1662,3c,2,189980,흥국에프엔비,8.597290e+09,2015,3.922360e+09,2016,4.235390e+09,2017
1663,3c,2,037440,희림,6.954140e+09,2015,5.398000e+09,2016,5.009800e+09,2017


In [15]:
profit_codes = df_all.code                   # 3 개년 모두 흑자인 회사 코드

In [16]:
len(set(profit_codes))                          #3 개년 모두 흑자인 회사 갯수

958

In [17]:
len(set(df_all[(df_all['smarket'] == '1')]['code']))     # 3 개년 모두 흑자인 유가증권시장 회사 갯수

450

In [19]:
len(set(df_all[(df_all['smarket'] == '2')]['code']))     # 3 개년 모두 흑자인 코스닥시장 회사 갯수

508

In [20]:
app = QApplication(sys.argv)
kiwoom = Kiwoom()
kiwoom.comm_connect()

connected


In [21]:
def price(code):
    kiwoom.set_input_value("종목코드", code)
    kiwoom.comm_rq_data("opt10001_req", "opt10001", 0, "2000")
    highest_250 = kiwoom.highest_250
    lowest_250 = kiwoom.lowest_250
    current_price = kiwoom.current_price
    
    return (highest_250, lowest_250, current_price)

In [22]:
df_all['highest_250'] = np.nan
df_all['lowest_250']  = np.nan
df_all['current_price'] = np.nan

In [23]:
for k, code in profit_codes[:50].items():
    time.sleep(TR_REQ_TIME_INTERVAL)
    
    highest_250, lowest_250, current_price = price(code)
    
    if highest_250 < 0:
        highest_250 *= -1
    if lowest_250 < 0:
        lowest_250 *= -1
    if current_price < 0:
        current_price *= -1
    
    if current_price <= lowest_250 * 1.2:
        
        df_all.loc[k, 'highest_250'] = highest_250
        df_all.loc[k, 'lowest_250'] = lowest_250
        df_all.loc[k, 'current_price'] = current_price
        name = df_all.loc[k, 'code_name']
        
        print(code, name, highest_250, lowest_250, current_price, '\n')

001040 CJ 226500 153500 159000 

037560 CJ헬로비전 10800 6600 7580 

155660 DSR 14300 4805 5680 

078930 GS 77600 56300 59800 

028150 GS홈쇼핑 250400 176000 177100 

024120 KB오토시스 8900 5890 5930 

036030 KTH 7500 5070 5930 

060370 KT서브마린 5300 4200 4740 

001120 LG상사 33850 25450 26450 

032640 LG유플러스 17900 11700 11900 

011070 LG이노텍 188000 115500 137000 

108670 LG하우시스 116000 75400 79200 

000050 경방 17050 12950 14200 

013580 계룡건설산업 23950 15300 17100 

000270 기아자동차 40350 29950 31600 

004270 남성 2790 1855 1940 

090350 노루페인트 9670 7760 7870 

000320 노루홀딩스 19250 14050 14850 

004370 농심 380000 292000 311000 

054050 농우바이오 18200 14450 16050 

054050 농우바이오 18200 14450 16050 

064260 다날 6940 4410 5020 

019680 대교 8770 7630 7740 

000210 대림산업 94600 70000 78400 

005710 대원산업 9000 6700 7230 

100130 동국S&C 7710 4455 4750 

001230 동국제강 15850 9120 9190 

001520 동양 2520 1700 1885 

084670 동양고속 44700 24200 27000 

000150 두산 149500 94200 108000 

034020 두산중공업 25050 14300 15250 

003160 디아이 7900 4150 4870 



In [24]:
df_all.dropna(inplace=True)

In [30]:
len(set(df_all['code']))     # 3 개년 모두 영업이익 흑자이면서  현재가가 52주 저가 * 1.2 이하인 회사 갯수

34

In [26]:
len(set(df_all[(df_all['smarket'] == '1')]['code']))     # 3 개년 모두 영업이익 흑자이면서  현재가가 52주 저가 * 1.2 인 유가증권시장 회사 갯수

25

In [27]:
len(set(df_all[(df_all['smarket'] == '2')]['code']))     # 3 개년 모두 영업이익 흑자이면서  현재가가 52주 저가 * 1.2 인 코스닥시장 회사 갯수

9

In [28]:
df_all[(df_all['smarket'] == '1')]     # 유가증권시장

,kind,smarket,code,code_name,amount_x,year_x,amount_y,year_y,amount,year,highest_250,lowest_250,current_price
0,2,1,001040,CJ,5.287360e+10,2015,5.880760e+10,2016,6.086360e+10,2017,226500.0,153500.0,159000.0
2,2,1,037560,CJ헬로비전,1.049700e+11,2015,4.293650e+10,2016,5.522610e+10,2017,10800.0,6600.0,7580.0
3,2,1,155660,DSR,1.183600e+10,2015,1.335450e+10,2016,1.236020e+10,2017,14300.0,4805.0,5680.0
4,2,1,078930,GS,1.444180e+11,2015,3.358060e+11,2016,4.305970e+11,2017,77600.0,56300.0,59800.0
11,2,1,001120,LG상사,7.442510e+10,2015,2.649260e+10,2016,4.134470e+10,2017,33850.0,25450.0,26450.0
13,2,1,032640,LG유플러스,6.463320e+11,2015,7.557770e+11,2016,6.354370e+11,2017,17900.0,11700.0,11900.0
14,2,1,011070,LG이노텍,1.841350e+11,2015,7.129730e+10,2016,1.346580e+11,2017,188000.0,115500.0,137000.0
15,2,1,108670,LG하우시스,1.175470e+11,2015,1.256580e+11,2016,9.742600e+10,2017,116000.0,75400.0,79200.0
18,2,1,000050,경방,3.862990e+10,2015,4.257680e+10,2016,2.768210e+10,2017,17050.0,12950.0,14200.0
19,2,1,013580,계룡건설산업,2.927910e+10,2015,6.095110e+10,2016,5.040840e+10,2017,23950.0,15300.0,17100.0


In [29]:
df_all[(df_all['smarket'] == '2')]     # 코스닥시장

,kind,smarket,code,code_name,amount_x,year_x,amount_y,year_y,amount,year,highest_250,lowest_250,current_price
6,2,2,028150,GS홈쇼핑,1.124660e+11,2015,1.286310e+11,2016,1.055400e+11,2017,250400.0,176000.0,177100.0
7,2,2,024120,KB오토시스,1.138540e+10,2015,1.516700e+10,2016,1.022070e+10,2017,8900.0,5890.0,5930.0
8,2,2,036030,KTH,5.348240e+09,2015,6.258290e+09,2016,3.515490e+09,2017,7500.0,5070.0,5930.0
9,2,2,060370,KT서브마린,6.173340e+09,2015,8.068860e+09,2016,9.328760e+09,2017,5300.0,4200.0,4740.0
28,2,2,054050,농우바이오,8.098150e+08,2015,1.106450e+10,2016,8.875700e+09,2017,18200.0,14450.0,16050.0
29,2,2,054050,농우바이오,1.273650e+10,2015,1.106450e+10,2016,8.875700e+09,2017,18200.0,14450.0,16050.0
30,2,2,064260,다날,9.573460e+09,2015,1.642110e+10,2016,1.460740e+10,2017,6940.0,4410.0,5020.0
35,2,2,005710,대원산업,1.466470e+10,2015,1.672490e+10,2016,2.563160e+10,2017,9000.0,6700.0,7230.0
36,2,2,100130,동국S&C,1.510610e+10,2015,2.175880e+10,2016,1.345310e+10,2017,7710.0,4455.0,4750.0
44,2,2,066900,디에이피,1.244710e+10,2015,4.939260e+09,2016,5.003360e+09,2017,5400.0,2840.0,3250.0
